In [ ]:
import pandas as pd

In [ ]:
#harom tablazat: repuloterek listaja, szomszedos orszagok adatbazisa minden orszagnal es egy tablaba amibe outputot gyujtunk
repuloterek = pd.read_excel('airports.xlsx')
szomszedok=pd.read_excel('GEODATASOURCE-COUNTRY-BORDERS.xlsx')
tablazat=pd.DataFrame(columns=['Repter','Varos','Orszag','Repterkod','Tavolsag','Latitude','Longitude'])

In [ ]:
#ez a resz csak a koordinatakat szedi ki a kiindulasi pontunknak (latitude es longitude valtozo)
import urllib.request
from urllib.error import HTTPError
import json

bingMapsKey = "Anwh6f1FKb8CIJbqi4Z9WEiHzV5LNB3L0JVrk85oZ6yEEpvL4k0nE3D4C8kEorfo"

Varos=input("Origin city:")
Orszag=input("Origin country:")

origin = Varos+', '+Orszag

encodedOrig = urllib.parse.quote(origin, safe='')

destination = Varos+', '+Orszag

encodedDest = urllib.parse.quote(destination, safe='')

routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Transit?wp.0=" + encodedOrig + "&wp.1=" + encodedDest + "&timeType=Departure&dateTime=3:00:00PM&output=json" + "&key=" + bingMapsKey
            #http://dev.virtualearth.net/REST/V1/Routes?wp.0=37.779160067439079,-122.42004945874214&wp.1=32.715685218572617,-117.16172486543655&key=BingMapsKey 

request = urllib.request.Request(routeUrl)
response = urllib.request.urlopen(request)

r = response.read().decode(encoding="utf-8")
result = json.loads(r)

latitude=result["resourceSets"][0]["resources"][0]['bbox'][0]
longitude=result["resourceSets"][0]["resources"][0]['bbox'][1]

In [ ]:
#elkuszul a 'tablazat' nevu adattabla amibe a od felsorolja a kiinulasi pontunktol a legkozelebbi reptereket (max 5 perc meg lefut, kiindulasi hely fuggo, Budapesttel egy perc)
#sajat orszagban szomszedos orszagokat vizsgalja, de csak 
szomszedosorszagok=[]
for i in range(len(szomszedok)):
    if Orszag==szomszedok.iloc[:,1][i]:
        szomszedosorszagok.append(szomszedok.iloc[:,3][i])

for i in range(len(repuloterek)):
    try:
        for j in range(len(szomszedosorszagok)):
            if repuloterek.iloc[:,2][i]==szomszedosorszagok[j] or repuloterek.iloc[:,2][i]==Orszag:
                if latitude-3<float(repuloterek.iloc[:,4][i]) and float(repuloterek.iloc[:,4][i])<latitude+3:
                    if longitude-3<float(repuloterek.iloc[:,5][i]) and float(repuloterek.iloc[:,5][i])<longitude+3:
                        origin = Varos+', '+Orszag

                        encodedOrig = urllib.parse.quote(origin, safe='')

                        destination = repuloterek.iloc[:,0][i]+", "+repuloterek.iloc[:,1][i]+", "+repuloterek.iloc[:,2][i]

                        encodedDest = urllib.parse.quote(destination, safe='')

                        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Transit?wp.0=" + encodedOrig + "&wp.1=" + encodedDest + "&timeType=Departure&dateTime=3:00:00PM&output=json" + "&key=" + bingMapsKey
                                    #http://dev.virtualearth.net/REST/V1/Routes?wp.0=37.779160067439079,-122.42004945874214&wp.1=32.715685218572617,-117.16172486543655&key=BingMapsKey 

                        request = urllib.request.Request(routeUrl)
                        response = urllib.request.urlopen(request)

                        r = response.read().decode(encoding="utf-8")
                        result = json.loads(r)

                        itineraryItems = result["resourceSets"][0]["resources"][0]["routeLegs"][0]["itineraryItems"]

                        #print(result["resourceSets"][0]["resources"][0]["travelDuration"])
                        ujsor={'Repter':repuloterek.iloc[:,0][i],'Varos':repuloterek.iloc[:,1][i],'Orszag':repuloterek.iloc[:,2][i],'Repterkod':repuloterek.iloc[:,3][i],'Tavolsag':result["resourceSets"][0]["resources"][0]["travelDuration"],'Latitude':repuloterek.iloc[:,4][i],'Longitude':repuloterek.iloc[:,5][i]}
                        tablazat=tablazat.append(ujsor,ignore_index=True)
                        if repuloterek.iloc[:,2][i]==Orszag:
                            break

    except urllib.error.HTTPError:
        pass

In [ ]:
#leszedi az elso ot repuloteret, a szerint hogy mennyi ido eljutni rajuk, ezeket fogja a kovetkezo scrapben futtatni
repterkodok=[]
for i in range(5):    
    repterkodok.append(tablazat.sort_values(by='Tavolsag').reset_index(drop=True).iloc[:,3][i])

In [ ]:
#ez scrapeli a repulesi menetrendet, csak kozvetlen return jaratokat nez, a kodban van a destinations valtozo,
#ide a repter IATA kodjat kell megadni es lista, szoval lehet tobbet is, aztan a startdates az utazas idopontja
#ide is lehet tobbet megadni,majd a stayingdays (int) az hogy mennyit akarunk maradni a visszaut keresese miatt
#letrehozza a results tablazatot amibe benne lesz az osszes lehetseges repulout
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

def scrape(origin, destination, startdate, days, requests):
    
    global results
    
    enddate = datetime.strptime(startdate, '%Y-%m-%d').date() + timedelta(days)
    enddate = enddate.strftime('%Y-%m-%d')

    url = "https://www.kayak.com/flights/" + origin + "-" + destination + "/" + startdate + "/" + enddate + "?sort=bestflight_a&fs=stops=0"
    print("\n" + url)

    chrome_options = webdriver.ChromeOptions()
    agents = ["Chrome/73.0.3683.68"]
    print("User agent: " + agents[(requests%len(agents))])
    chrome_options.add_argument('--user-agent=' + agents[(requests%len(agents))] + '"')    
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome('/Users/macbookair/Downloads/chromedriver', options=chrome_options, desired_capabilities=chrome_options.to_capabilities())
    driver.implicitly_wait(20)
    driver.get(url)

    #Check if Kayak thinks that we're a bot
    time.sleep(5) 
    soup=BeautifulSoup(driver.page_source, 'lxml')

    if soup.find_all('p')[0].getText() == "Please confirm that you are a real KAYAK user.":
        print("Kayak thinks I'm a bot, which I am ... so let's wait a bit and try again")
        driver.close()
        time.sleep(20)
        return "failure"

    time.sleep(5) #wait 20sec for the page to load
    
    soup=BeautifulSoup(driver.page_source, 'lxml')
    
    #get the arrival and departure times
    deptimes = soup.find_all('span', attrs={'class': 'depart-time base-time'})
    arrtimes = soup.find_all('span', attrs={'class': 'arrival-time base-time'})
    meridies = soup.find_all('span', attrs={'class': 'time-meridiem meridiem'})
    
    deptime = []
    for div in deptimes:
        deptime.append(div.getText()[:-1])    
        
    arrtime = []
    for div in arrtimes:
        arrtime.append(div.getText()[:-1])   

    meridiem = []
    for div in meridies:
        meridiem.append(div.getText())
        
    deptime = np.asarray(deptime)
    deptime = deptime.reshape(int(len(deptime)/2), 2)
    
    arrtime = np.asarray(arrtime)
    arrtime = arrtime.reshape(int(len(arrtime)/2), 2)      
    
    meridiem = np.asarray(meridiem)
    meridiem = meridiem.reshape(int(len(meridiem)/4), 4)
        
    #Get the price
    regex = re.compile('Common-Booking-MultiBookProvider (.*)multi-row Theme-featured-large(.*)')
    price_list = soup.find_all('div', attrs={'class': regex})
    
    price = []
    for div in price_list:
        price.append(int(div.getText().split('\n')[3][1:]))

    df = pd.DataFrame({"origin" : origin,
                       "destination" : destination,
                       "startdate" : startdate,
                       "enddate" : enddate,
                       "price": price,
                       "currency": "USD",
                       "deptime_o": [m+str(n) for m,n in zip(deptime[:,0],meridiem[:,0])],
                       "arrtime_d": [m+str(n) for m,n in zip(arrtime[:,0],meridiem[:,1])],
                       "deptime_d": [m+str(n) for m,n in zip(deptime[:,1],meridiem[:,2])],
                       "arrtime_o": [m+str(n) for m,n in zip(arrtime[:,1],meridiem[:,3])]
                       })

    results = pd.concat([results, df], sort=False)

    driver.close() #close the browser

    time.sleep(5) #wait 15sec until the next request
    
    return "success"

#Create an empty dataframe 
results = pd.DataFrame(columns=['origin','destination','startdate','enddate','deptime_o','arrtime_d','deptime_d','arrtime_o','currency','price'])

for x in range(len(repterkodok)):
    requests = 0 

    origin = repterkodok[x]
    destinations = ['GVA']
    startdates = ['2020-10-16']
    stayingdays=3

    for destination in destinations:
        for startdate in startdates:   
            requests = requests + 1
            while scrape(origin, destination, startdate, stayingdays, requests) != "success":
                requests = requests + 1


In [ ]:
#egyetlen rendezesi elve az ar, ugy nezi az elso talalatot
results=results.sort_values(by='price').reset_index(drop=True)

In [ ]:
#utvonalat tervez a kiindulasi helyunkrol a kiindulasi repterunkre, megadja annak menetet es menetidejet majd a repuloutank adatait
for i in range(len(repuloterek)):
    
    if repuloterek.iloc[:,3][i]==results.iloc[0][0]:    
        origin = Varos+', '+Orszag

        encodedOrig = urllib.parse.quote(origin, safe='')
        
        destlat = repuloterek.iloc[:,4][i]
        destlon = repuloterek.iloc[:,5][i]

        routeUrl = "http://dev.virtualearth.net/REST/V1/Routes/Transit?wp.0=" + encodedOrig + "&wp.1=" + str(destlat) + "," + str(destlon) + "&timeType=Departure&dateTime=3:00:00PM&output=json" + "&key=" + bingMapsKey
                    #http://dev.virtualearth.net/REST/V1/Routes?wp.0=37.779160067439079,-122.42004945874214&wp.1=32.715685218572617,-117.16172486543655&key=BingMapsKey 

        request = urllib.request.Request(routeUrl)
        response = urllib.request.urlopen(request)

        r = response.read().decode(encoding="utf-8")
        result = json.loads(r)

        itineraryItems = result["resourceSets"][0]["resources"][0]["routeLegs"][0]["itineraryItems"]

        print("Time to Airport: "+str(int(result["resourceSets"][0]["resources"][0]["travelDuration"]/60))+' minutes')
        print('Directions to Airport:')
        for item in itineraryItems:
            print(item["instruction"]["text"])
        
        print('Your flight:')
        print(results.iloc[0])
